In [1]:
import numpy as np
import time
n=100

kernel = np.arange(n**2).reshape((n,n))
print(kernel)

[[   0    1    2 ...,   97   98   99]
 [ 100  101  102 ...,  197  198  199]
 [ 200  201  202 ...,  297  298  299]
 ..., 
 [9700 9701 9702 ..., 9797 9798 9799]
 [9800 9801 9802 ..., 9897 9898 9899]
 [9900 9901 9902 ..., 9997 9998 9999]]


In [2]:
def conv0(K):
    #naive method
    n=len(K)
    result= np.zeros((n,n,n,n))
    for k in range(n):
        for l in range(n):
            for i in range(n):
                for j in range(n):
                    result[i,j,k,l] = K[i-k,j-l] if ((i-k)>=0 and (j-l)>=0) else 0
    return result

def conv1(K):
    #slightly clever method
    n=len(K)
    a=np.array([[1 if j-i ==-1 else 0 for j in range(n)] for i in range(n)])
    aT=a.T

    halfshiftedkernels=[K]
    hsk = K
    for k in range(n):
        hsk = np.dot(a,hsk)
        halfshiftedkernels += [hsk]
    halfshiftedkernels = np.array(halfshiftedkernels)

    shiftedkernels = [halfshiftedkernels]
    sk= halfshiftedkernels
    for l in range(n):
            sk = np.dot(sk,aT)
            shiftedkernels += [sk]
    return np.transpose(np.array(shiftedkernels),(2,3,1,0))

def preconv(n):
    # auxiliary function to make upper triangular nxnxn matrix
    a=np.array([[1 if (j-i ==-1) else 0 for j in range(n)] for i in range(n)])
    
    hsk=np.eye(n,n)
    halfshiftedkernels=[hsk]
    for k in range(n):
        hsk = np.dot(a,hsk)
        halfshiftedkernels += [hsk]
    A = np.array(halfshiftedkernels)
    
    return A

def conv2(K):
    # cleverder method
    t=time.time()
    A = preconv(len(K)) #this step is recyclable for same dimensions
    print('auxtime:'+str(time.time()-t))
    AT = np.transpose(A,(1,2,0))
    result = np.dot(np.dot(A,K),AT)
    return np.transpose(result,(1,2,0,3))        


In [3]:
t=time.time()
print(conv0(kernel)[4,4])
print(time.time()-t)

t=time.time()
print(conv1(kernel)[4,4])
print(time.time()-t)

t=time.time()
print(conv2(kernel)[4,4])
print(time.time()-t)

[[ 404.  403.  402. ...,    0.    0.    0.]
 [ 304.  303.  302. ...,    0.    0.    0.]
 [ 204.  203.  202. ...,    0.    0.    0.]
 ..., 
 [   0.    0.    0. ...,    0.    0.    0.]
 [   0.    0.    0. ...,    0.    0.    0.]
 [   0.    0.    0. ...,    0.    0.    0.]]
24.863622903823853
[[404 403 402 ...,   0   0   0]
 [304 303 302 ...,   0   0   0]
 [204 203 202 ...,   0   0   0]
 ..., 
 [  0   0   0 ...,   0   0   0]
 [  0   0   0 ...,   0   0   0]
 [  0   0   0 ...,   0   0   0]]
4.386164903640747
auxtime:0.012032270431518555
[[ 404.  403.  402. ...,    0.    0.    0.]
 [ 304.  303.  302. ...,    0.    0.    0.]
 [ 204.  203.  202. ...,    0.    0.    0.]
 ..., 
 [   0.    0.    0. ...,    0.    0.    0.]
 [   0.    0.    0. ...,    0.    0.    0.]
 [   0.    0.    0. ...,    0.    0.    0.]]
8.558761596679688
